# Simple YOLO model

Вся выжимка из анализа набора данных при помощи "пристального взгляда" описана в ```README.md```. Также там есть размышления по решению данной задачи через **эвристику**.

## Задача
Определить количество книг на фотографии.

## Размышления
Первое, что мне пришло в голову исходя из глобальной задачи (решить проблему "Правила размещения на площадке") - это применить для этой задачи **детекторы**. Т.е. в дальнейшем хотим использовать данную модель/систему для разбиения одного размещения с несколькими книгами на несколько размещений по одной книге. 

Почему именно детекторы - потому что мы можем увидеть, что именно модель/система считает за книгу, также есть пул предобученных моделей. 

## Логика
1) Беру предобученную **YOLO** для детекции (Предобученный детектор можно брать любой, главное, чтобы в датасете, на котором он обучался, был класс "книга")
2) Прогоняем фото через детектор 
3) Ищем детекции с классом соответствющим книге
4) Считаем кол-во детекций (кол-во детекций = кол-ву книг на фото)

На данном этапе я пока **не буду** делать препроцессинг и постпроцессинг.

Буду считать данное решение **baseline**.

# Import

In [4]:
import os

while os.getcwd().split("/")[-1] != "book_counter":
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [5]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

# Input/Output Configuration 

In [ ]:
images_folder = 'task_images'
image_files = [f for f in os.listdir(images_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
image_files.sort(key=lambda x: int(x.split('.')[0]))

In [21]:
output_folder = 'detection_results_simple_yolo'

os.makedirs(output_folder, exist_ok=True)

In [36]:
solution_csv_path = "solution_simple_yolo.csv"

# Init model

In [37]:
model = YOLO('yolo11s.pt')
conf_threshold = 0.5

# Inference (Object Detection and Counting)

In [38]:
submission_df = pd.DataFrame(columns=['image_id', 'number_of_books'])

In [39]:
for img_file in image_files:
    img_path = os.path.join(images_folder, img_file)
    image = cv2.imread(img_path)

    image_with_boxes = image.copy()
    book_count = 0

    results = model(image, conf=conf_threshold, verbose=False)
    for result in results:
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            class_ids = result.boxes.cls.cpu().numpy().astype(int)

            for i, (box, conf, cls_id) in enumerate(zip(boxes, confidences, class_ids)):
                if cls_id == 73:
                    book_count += 1
                    x1, y1, x2, y2 = map(int, box)
                    cv2.rectangle(
                        image_with_boxes, (x1, y1), (x2, y2), (0, 255, 0), 2
                    )
                    label = f"{conf:.2f}"
                    (text_width, text_height), baseline = cv2.getTextSize(
                        label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2
                    )
                    cv2.rectangle(
                        image_with_boxes,
                        (x1, y1 - text_height - 5),
                        (x1 + text_width, y1),
                        (0, 255, 0),
                        -1,
                    )
                    cv2.putText(
                        image_with_boxes,
                        label,
                        (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.7,
                        (0, 0, 0),
                    )

    output_path = os.path.join(output_folder, f"{img_file.split('.')[0]}_result.jpg")
    cv2.imwrite(output_path, image_with_boxes)
    submission_df = pd.concat(
        [submission_df, pd.DataFrame({"image_id": [img_file.split('.')[0]], "number_of_books": [book_count]})],
        ignore_index=True,
    )

In [40]:
submission_df.to_csv(
    solution_csv_path, 
    header=['image_id', 'number_of_books'], 
    index=False
)